In [1]:
import yaml
import pandas as pd

folder_location = 'C:/Users/ttrol/CodingProjects/MatNet/scripts/webscraper/results_files/'
filename = 'AISI_steels.yaml'
file_location = folder_location + filename

with open(file_location, "r") as stream:
    try:
        raw_materials = yaml.safe_load(stream)
        print('Raw Size: ', len(raw_materials))
        raw_df = pd.DataFrame(raw_materials)
    except yaml.YAMLError as exc:
        print(exc)

Raw Size:  1813


In [19]:
import time

def time_it(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function '{func.__name__}' executed in {execution_time:.6f} seconds.")
        return result
    return wrapper

In [34]:
import requests
import random
from pymongo import MongoClient
from bs4 import BeautifulSoup

#oxylabs authentication
USERNAME = "pcho69"
PASSWORD = "StealthyWebsitePumpk1n"
ENDPOINT = "pr.oxylabs.io:7777"

#connect to mongodb client
client = MongoClient("mongodb://localhost:27017/")
db = client.matjet
collection = db.materials


headers =  {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
}

entry = ('http://customer-%s:%s@%s' %
    (USERNAME, PASSWORD, ENDPOINT))
query = {
    'http': entry,
    'https': entry,
}
session = requests.Session()


# link = raw_df.link[random.randint(0,len(raw_df)-1)]
link = raw_df.link[0]

# for i in range(1):
@time_it
def parse_page(link):
    response = session.get(
        headers=headers,
        url=link,
        # proxies= query,
        verify=True
    )

    soup = BeautifulSoup(response.content,'lxml')

    table = soup.select('div#ctl00_ContentMain_ucDataSheet1_pnlMaterialData table.tabledataformat')
    print(len(table))
    if len(table) != 2:
        raise Exception('IP Blocked')

    #find supplementary notes in tables
    matl_notes_table = table[0].find(id= "ctl00_ContentMain_ucDataSheet1_trMatlNotes")
    category_table = table[0].find(id= "ctl00_ContentMain_ucDataSheet1_trMatlGroups")
    
    material_notes = None
    categories = None

    try:
        if matl_notes_table is not None:
            material_notes = matl_notes_table.find('td').text   #type: ignore
            print('Notes: ', material_notes)
        
        if category_table is not None:
            categories = category_table.find('td').text  #type: ignore
            categories = [item.strip() for item in categories.split(";")]
            print('Categories:', categories)
    except Exception:
        exception_msg = 'Issues with grabbing material and category notes: ' \
            + link
        print(exception_msg)
        

    #Second table in html page has correct content
    main_table_rows = table[1].find_all('tr')
    property_tables = []
    if main_table_rows is None:
        raise TypeError('Cannot parse None type')
  
    data = []
    
    #This creates a list of tables from the main content table. 
    for row in main_table_rows:
        cols = row.find_all('td')

        #necessary to delimit tables within content tables
        if not cols:
            data = []
            cols = row.find_all('th')
            property_tables.append(data)

        #Remove whitespace in between
        cols = [ele.text.strip() for ele in cols]
        
        # Get rid of empty values
        data.append([ele for ele in cols if ele]) 
    
    property_tables.append(data)
    
    #Used for removing all the empty rows in each individual table
    for i in range(len(property_tables)):
        property_tables[i] = [x for x in property_tables[i] if x != []]

    #find name of material in page and remove unnecessary whitespace
    material_name = soup.find('title').get_text() # pyright: ignore[reportOptionalMemberAccess]  # noqa: E501
    material_name = material_name.strip()

    return {'name': material_name,'notes': material_notes, \
            'categories': categories, 'properties': property_tables}

result = parse_page(link)
result


2
Notes:  Normalized at 925°C in 32 mm rounds, machined to 25 mm rounds, pseudocarburized at 925°C for 8 hours, oil quenched, 150°C temper, tested in 12.8 mm rounds
Categories: ['Metal', 'Ferrous Metal', 'Alloy Steel', 'AISI 9000 Series Steel', 'Low Alloy Steel', 'Carbon Steel', 'Low Carbon Steel']
Function 'parse_page' executed in 2.181553 seconds.


{'name': 'AISI E9310H Steel, normalized, machined to 25 mm (1 in.) rounds, pseudocarburized, 150°C (300°F) temper',
 'notes': 'Normalized at 925°C in 32 mm rounds, machined to 25 mm rounds, pseudocarburized at 925°C for 8 hours, oil quenched, 150°C temper, tested in 12.8 mm rounds',
 'categories': ['Metal',
  'Ferrous Metal',
  'Alloy Steel',
  'AISI 9000 Series Steel',
  'Low Alloy Steel',
  'Carbon Steel',
  'Low Carbon Steel'],
 'properties': [[['Physical Properties', 'Metric', 'English', 'Comments'],
   ['Density', '7.85 g/cc', '0.284 lb/in³']],
  [['Mechanical Properties', 'Metric', 'English', 'Comments'],
   ['Hardness, Brinell', '363', '363'],
   ['Hardness, Knoop', '392', '392', 'Converted from Brinell'],
   ['Hardness, Rockwell B', '100', '100', 'Converted from Brinell'],
   ['Hardness, Rockwell C',
    '40',
    '40',
    'Converted from Brinell hardness value.'],
   ['Hardness, Vickers',
    '384',
    '384',
    'Converted from Brinell hardness value.'],
   ['Tensile Streng